In [ ]:
import numpy as np
import pandas as pd
import yaml
import json
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, mean_absolute_error
from tensorflow.keras.models import model_from_json

from tqdm import tqdm
import os
import sys
import warnings
from dotenv import load_dotenv

warnings.filterwarnings("ignore")
load_dotenv()
REPO_PATH = os.getenv("REPO_PATH")
sys.path.insert(0, rf'{REPO_PATH}src_HF')

from utils.forecast_utils import load_prepared_data, preprocess_data, build_rnn_model

### Load data

In [ ]:
# Specify data
future = 'CLc1'
topic = 'CEN'

# Load data
df = load_prepared_data(future, topic)

# load yaml file
with open('variable_config.yaml') as file:
    config = yaml.load(file, Loader=yaml.FullLoader)

# Define feature list and target
feature_columns = list()

feature_columns += config['BASE']
feature_columns += config['SENTIMENT']
feature_columns += config['TEMPORAL']

print(feature_columns)

target_column = 'TARGET'

# Specify RNN model. Alternatives: BiLSTM, GRU or BiGRU
rnn_type='LSTM'

# Define window size
window_size = 30

# Scaling and splitting into test and train data sets. Default train/val size = 80%
train_generator, val_generator, test_generator = preprocess_data(
    df, 
    feature_columns, 
    target_column, 
    window_size,
    test_size=0.2,
    val_size=0.2
)


### Load models

In [ ]:
MODEL_NAMES = [
    'CLc1_CEN_LSTM_2024.05.14_17.18'
]

model_dict = dict()

for model_name in tqdm(MODEL_NAMES):
    with open(f'model_archive/{model_name}_info.json') as f:
        model_info = json.load(f)
    with open(f'model_archive/{model_name}_model.json') as f:
        loaded_model_json = f.read()

    model = model_from_json(loaded_model_json)
    print(type(model)) 

    model.load_weights(f'model_archive/{model_name}.h5')

    model_components = model_name.split('_')

    test_predictions = model.predict(test_generator)
    test_predictions = test_predictions.flatten()

    test_targets = np.concatenate([y for _, y in test_generator])
    test_targets = test_targets.flatten()

    mse = mean_squared_error(test_targets, test_predictions)
    mae = mean_absolute_error(test_targets, test_predictions)

    model_dict[model_name] = {
        'model': model,
        'test_predictions': test_predictions,
        'test_targets': test_targets,
        'mse': mse,
        'mae': mae
    }



In [ ]:
# Evaluate the model
view = 500



print(f'Mean Squared Error: {mse}')
print(f'Mean Absolute Error: {mae}')

# plot fit vs actual
plt.figure(figsize=(10, 5))
plt.plot(test_targets[-view:], label='Actual')
plt.plot(test_predictions[-view:], label='Predicted')
plt.title('Model Fit vs Actual')
plt.xlabel('Samples')
plt.ylabel('Price')
plt.legend()